### Library Import

In [1]:
import geopandas as gpd
from sentinelsat.sentinel import (
    InvalidChecksumError,
    SentinelAPIError,
)
from pathlib import Path
import rioxarray
from sentinelsat.sentinel import (
    SentinelAPI,
    geojson_to_wkt,
    read_geojson,
)
import sys
import xarray as xr
import datetime
from shapely.geometry import Polygon

### Vector File generation based on bounding box coordinates of AOI

In [2]:
def vector_file(coordinates):
    polygon_geom = Polygon(coordinates)
    crs = {'init': 'epsg:4326'}
    polygon = gpd.GeoDataFrame(index=[0], crs=crs, geometry=[polygon_geom])
    return polygon

In [4]:
place1_coord =( [76.8,28.4],[ 76.8,28.9],[77.4,28.9],[77.4,28.4])
place1_vector_file = vector_file(place1_coord)

In [5]:
place1_vector_file.to_file("D:\shp\DelhiBB.geojson", driver="GeoJSON")

### Sentinel hub(S5phub) user authentication 

In [2]:
DHUS_USER = "s5pguest"
DHUS_PASSWORD = "s5pguest"
DHUS_URL = "https://s5phub.copernicus.eu/dhus"

In [3]:
api = SentinelAPI(DHUS_USER, DHUS_PASSWORD, DHUS_URL)

### Date range

In [5]:
start_date = datetime.datetime(2021,5,24)
end_date = datetime.datetime(2021,5,27)
date = (start_date,end_date)

### Adding Query
    Productstype:     
    #   L2__O3____
    #   L2__NO2___
    #   L2__SO2___
    #   L2__CO____
    #   L2__CH4___
    #   L2__HCHO__
    #   L2__AER_AI
    #   L2__CLOUD_
 

In [6]:
query_body = {
        "date": date,
        "platformname": "Sentinel-5 Precursor",
        "producttype": 'L2__O3____'
}

In [7]:
footprint = geojson_to_wkt(read_geojson(Path('D:\shp\DelhiBB.geojson')))

In [8]:
products = api.query(footprint,**query_body)

In [9]:
products

OrderedDict([('8c849194-f955-4378-a09f-467da82f47dc',
              {'title': 'S5P_OFFL_L2__O3_____20210526T081020_20210526T095150_18741_01_020104_20210528T020223',
               'link': "https://s5phub.copernicus.eu/dhus/odata/v1/Products('8c849194-f955-4378-a09f-467da82f47dc')/$value",
               'link_alternative': "https://s5phub.copernicus.eu/dhus/odata/v1/Products('8c849194-f955-4378-a09f-467da82f47dc')/",
               'link_icon': "https://s5phub.copernicus.eu/dhus/odata/v1/Products('8c849194-f955-4378-a09f-467da82f47dc')/Products('Quicklook')/$value",
               'summary': 'Date: 2021-05-26T08:31:55Z, Instrument: TROPOMI, Satellite: Sentinel-5 Precursor, Size: 317.89 MB',
               'ondemand': 'false',
               'ingestiondate': datetime.datetime(2021, 5, 28, 12, 7, 26, 910000),
               'beginposition': datetime.datetime(2021, 5, 26, 8, 31, 55),
               'endposition': datetime.datetime(2021, 5, 26, 9, 30, 18),
               'generationdate': 

In [10]:
size=api.get_products_size(products)

In [12]:
len(products.keys())

4

In [36]:
ids_request = list(products.keys())

In [37]:
DOWNLOAD_DIR = 'D:\\Ozone\\Data\\'

In [38]:
ids_request 

['ae0c2dd4-a3eb-4b0e-a3c7-208e5491b84e',
 '4789ff6a-9b35-49da-b9b3-3cc52a36284c']

In [39]:
filenames = [
        DOWNLOAD_DIR + f"{products[file_id]['title']}.nc"
        for file_id in ids_request
    ]

In [40]:
filenames

['D:\\Ozone\\Data\\S5P_OFFL_L2__O3_____20210525T064750_20210525T082920_18726_01_020104_20210526T234243.nc',
 'D:\\Ozone\\Data\\S5P_OFFL_L2__O3_____20210524T070649_20210524T084819_18712_01_020104_20210526T003322.nc']

### Data Download

In [41]:
for i in range(len(ids_request)):
    api.download(ids_request[i],'D:\\Ozone\\Lockdown\\',checksum=True)

MD5 checksumming:   0%|          | 0.00/336M [00:00<?, ?B/s]

MD5 checksumming:   0%|          | 0.00/334M [00:00<?, ?B/s]

In [13]:
def Sentinel5p_download(download_path,start_date,end_date, aoi_path,Product):
    '''download_path : str
    start_date:timestamp
    end_date: timeStamp exclusive in date range
    aoi_path: str (geojson file path- Bounding box)
    Product:     #   L2__O3____
    #   L2__NO2___
    #   L2__SO2___
    #   L2__CO____
    #   L2__CH4___
    #   L2__HCHO__
    #   L2__AER_AI
    #   L2__CLOUD_
    
    '''
    DHUS_USER = "s5pguest"
    DHUS_PASSWORD = "s5pguest"
    DHUS_URL = "https://s5phub.copernicus.eu/dhus"
    api = SentinelAPI(DHUS_USER, DHUS_PASSWORD, DHUS_URL)
    date = (start_date,end_date)
    query_body = {
        "date": date,
        "platformname": "Sentinel-5 Precursor",
        "producttype": Product
    }
    footprint = geojson_to_wkt(read_geojson(Path(aoi_path)))
    products = api.query(footprint,**query_body)
    request_ids = list(products.keys())
    for i in range(len(request_ids)):
        api.download(ids_request[i],download_path,checksum=True)